In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Standard libraries
import logging
import sys
import re

import pandas as pd
import sqlalchemy as sa
import matplotlib as mpl
import matplotlib.pyplot as plt

import ast

import pudl
from pudl_ct.guts import *

mpl.style.use('dark_background')
pd.options.display.max_columns = None
plt.rcParams["figure.figsize"] = (15,10)

In [ ]:
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

### Make Shared Inputs
These tables will be shared between both the coal and gas outputs. This is especially important for CEMS because it takes a minute to process the hourly data

In [ ]:
# make the PUDL output object which will 
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
# the fill_net_gen arg is employing a beta feature!
# It will result in a much higher coverage of net generation
# and fuel consuption, but it has known issues with multi-fuel plants.
pudl_out = pudl.output.pudltabl.PudlTabl(
    pudl_engine,
    freq='AS',
    fill_fuel_cost=True,
    roll_fuel_cost=True,
    fill_net_gen=True,
    start_date='2017-01-01',
)

In [ ]:
%%time
# this takes several minutes.. it is aggregating the hourly cems records
epacems_path = pathlib.Path(pudl_settings['parquet_dir']) / 'epacems'
cems_by_boiler = get_cems(epacems_path)

In [ ]:
gen = (
    prep_gens_eia(pudl_out)
    .pipe(add_nems, pudl_out)
    .pipe(merge_gem_w_df, df_source='eia')
)

In [ ]:
eia_cems_merge = stuff(cems_by_boiler=cems_by_boiler, gen=gen, pudl_out=pudl_out)

### Make Outputs for Coal Modeling
These outputs will be aggregated to EIA/PUDL units

In [ ]:
# let's make the main outputs here...
out_pudl_units = make_ct_compilation(pudl_out, unit_id_col='unit_id_pudl')

### Make Outputs for Gas Modeling
These outputs will be aggregated to gem units.

In [ ]:
out_gem_units = make_ct_compilation(pudl_out, unit_id_col='unit_id_gem')

### Export the Various Outputs

In [ ]:
out_gem_units.to_csv(pathlib.Path.cwd().parent / 'outputs/gem_units.csv.gz')
out_pudl_units.to_csv(pathlib.Path.cwd().parent / 'outputs/eia_units.csv.gz')

eia_cems_merge.to_csv(pathlib.Path.cwd().parent / 'outputs/carbon-tracker-cems.csv.gz')

# there are the interim outputs... if your interested
gen.to_csv(pathlib.Path.cwd().parent / 'outputs/eia_generators.csv.gz')
#steam_df.to_csv(pathlib.Path.cwd().parent / 'outputs/ferc1_steam_records.csv.gz')
#steam_by_fuel.to_csv(pathlib.Path.cwd().parent / 'outputs/ferc1_steam_by_fuel.csv.gz')
cems_by_boiler.to_csv(pathlib.Path.cwd().parent / 'outputs/cems_by_boiler.csv.gz')

In [ ]:
gem_out.to_csv(pathlib.Path.cwd().parent / 'outputs/ct-plant-compilation-gem-id.csv.gz')
gem_out_alloc.to_csv(pathlib.Path.cwd().parent / 'outputs/ct-plant-compilation-allocated-gem-id.csv.gz')

### Plot some stuff

In [ ]:
out_df_pudl = coal_out[
    (coal_out.fuel_type_code_pudl == 'coal') 
    & (coal_out.report_date.dt.year.isin(NEMS_FILE_NAMES.keys()))
]


plt.hist(
    out_df_pudl.capex_annual_per_kw_avg_life,
    weights=out_df_pudl.capacity_mw,
    bins=50,
    label='FERC',
    color='blueviolet',
)
plt.hist(
    out_df_pudl.capex_annual_per_kw_nems,
    weights=out_df_pudl.capacity_mw,
    bins=50,
    label='NEMS',
    color='thistle'
)

plt.ylabel('Capacity (MW)')
plt.xlabel('Annual Capex ($/kW)')
plt.legend(fontsize=16)
plt.title('Annual Coal Capex from FERC and NEMS', fontsize=20)
plt.show();



out_df_gem = gas_out[
    (gas_out.fuel_type_code_pudl == 'gas') 
    & (gas_out.report_date.dt.year.isin(NEMS_FILE_NAMES.keys()))
]


plt.hist(
    out_df_gem.capex_annual_per_kw_avg_life,
    weights=out_df_gem.capacity_mw,
    bins=50,
    label='FERC',
    color='blueviolet',
)
plt.hist(
    out_df_gem.capex_annual_per_kw_nems,
    weights=out_df_gem.capacity_mw,
    bins=50,
    label='NEMS',
    color='thistle'
)

plt.ylabel('Capacity (MW)')
plt.xlabel('Annual Capex ($/kW)')
plt.legend(fontsize=16)
plt.title('Annual Gas Capex from FERC and NEMS', fontsize=20)
plt.show();

In [ ]:
colors_fuel = ['blueviolet', 'deeppink', 'yellow', 'lime', 'aqua', ]
fuels= [f for f in steam_df.fuel_type_code_pudl.unique() if f not in ['unknown', ''] and f is not np.NAN]

In [ ]:
for fuel, color in zip(fuels, colors_fuel):
    df = steam_df[steam_df.fuel_type_code_pudl == fuel]
    plt.hist(
        df.capex_annual_per_mw/1000,
        weights=df.capacity_mw,
        bins=100, range=(-10, 100),
        label=fuel,
        alpha=.7, color=color
    )
    
plt.title('Capital Additions per kW by Fuel', fontsize=18)
plt.xlabel('Capital Additons ($/kW)')
plt.ylabel('Weighted by Capacity (MW)')
plt.legend()
plt.show()

In [ ]:
for fuel, color in zip(fuels, colors_fuel):
    df = steam_df[steam_df.fuel_type_code_pudl == fuel]
    plt.hist(
        df.capex_annual_per_mw_rolling/1000,
        weights=df.capacity_mw,
        bins=100, range=(-10, 100),
        label=fuel,
        alpha=.7, color=color
    )
    
plt.title('Capital Additions per kW by Fuel', fontsize=18)
plt.xlabel('Capital Additons ($/kW)')
plt.ylabel('Weighted by Capacity (MW)')
plt.legend()
plt.show()

In [ ]:
for fuel, color in zip(fuels, colors_fuel):
    df = steam_df[steam_df.fuel_type_code_pudl == fuel]
    plt.hist(
        df.capex_annual_per_mwh,
        weights=df.net_generation_mwh,
        bins=100, range=(-10, 50),
        label=fuel,
        alpha=.7,
        color=color
    )
    
plt.title('Capital Additions per MWh by Fuel', fontsize=18)
plt.xlabel('Capital Additions $/MWh')
plt.ylabel('Weighted by Net Generation (MWh)')
plt.legend()
plt.show()

In [ ]:
for year in range(2009, 2020):
    df = steam_df[
        (steam_df.report_year == year)
        & (steam_df.fuel_type_code_pudl == 'coal')
    ]
    plt.hist(
        df.capex_annual_per_mwh,
        weights=df.net_generation_mwh,
        bins=100, range=(-10, 100),
        label=year,
        alpha=.7
    )
    
plt.title('Capital Additions per MWh by Year', fontsize=18)
plt.xlabel('Capital Additons per MWh')
plt.legend()
plt.show()

In [ ]:
for fuel in ['coal', 'gas']:
    plt.hist(
        gens_w_ferc1_nems[gens_w_ferc1_nems.fuel_type_code_pudl == fuel].fuel_cost_per_mmbtu,
        label=fuel,
        bins=50, range=(-2,10)
    )
    
plt.legend()
plt.show()

### Charting the different FERC methods

In [ ]:
eia_ferc_fuel = merge_eia_ferc_simple(unit=unit, steam_df=steam_df, steam_by_fuel=steam_by_fuel)
#eia_ferc_unit = merge_eia_ferc_unit(gen)

In [ ]:
for fuel in ['coal', 'gas']:
    fuel_df = eia_ferc_fuel[
        (eia_ferc_fuel.fuel_type_code_pudl == fuel)
        & (eia_ferc_fuel.report_date.dt.year == 2019)
    ]
    plt.hist(
        fuel_df.opex_nonfuel_per_mwh, 
        weights=fuel_df.net_generation_mwh,
        range=(-10,50),
        bins=100,
        label=fuel
    )
plt.legend()
plt.show()

In [ ]:
non_matching = gens_w_ferc1[
    (gens_w_ferc1.opex_nonfuel_per_mwh_plant_fuel !=
    gens_w_ferc1.opex_nonfuel_per_mwh_unit)
    & gens_w_ferc1.opex_nonfuel_per_mwh_plant_fuel.notnull()
    & gens_w_ferc1.opex_nonfuel_per_mwh_unit.notnull()
]

In [ ]:
fig, (ax) = plt.subplots(ncols=1, nrows=1, figsize=(10, 10))
ax.scatter(non_matching.opex_nonfuel_per_mwh_plant_fuel,
           non_matching.opex_nonfuel_per_mwh_unit,
           color='aquamarine', alpha=0.1, 
           #label=field
          )
lims = (1e0, 1e5)
ax.set_ylim(lims)
ax.set_xlim(lims)

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Opex Non-fuel ($/MWh) plant_fuel')
ax.set_ylabel('Opex Non-fuel ($/MWh) plant_unit')
ax.set_title(f"Non-Fuel Opex plant_fuel vs. plant_unit (Non-matching)", {'fontsize': 18,'fontweight' : 'bold'})

In [ ]:
fig, (ax) = plt.subplots(ncols=1, nrows=1, figsize=(10, 10))
for fuel_type in ['gas', 'coal']:
    non_matching_fuel = non_matching[non_matching.fuel_type_code_pudl == fuel_type]
    ax.scatter(non_matching_fuel.opex_nonfuel_per_mwh_plant_fuel,
               non_matching_fuel.opex_nonfuel_per_mwh_unit,
               #color='aquamarine',
               alpha=0.3, 
               label=fuel_type
              )
    lims = (1e0, 1e3)
ax.set_ylim(lims)
ax.set_xlim(lims)
ax.legend()

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Opex Non-fuel ($/MWh) plant_fuel')
ax.set_ylabel('Opex Non-fuel ($/MWh) plant_unit')
ax.set_title(f"Non-Fuel Opex plant_fuel vs. plant_unit (Non-matching)", {'fontsize': 18,'fontweight' : 'bold'})

In [ ]:
fig, (ax) = plt.subplots(ncols=1, nrows=1, figsize=(10, 10))
#for year in non_matching.report_date.sort_values().dt.year.unique():
for year in [2017,2018,2019]:
    non_matching_year = non_matching[non_matching.report_date.dt.year == year]
    ax.scatter(non_matching_year.opex_nonfuel_per_mwh_plant_fuel,
               non_matching_year.opex_nonfuel_per_mwh_unit,
               #color='aquamarine',
               alpha=0.3, 
               label=year
              )
    lims = (1e0, 1e5)
ax.set_ylim(lims)
ax.set_xlim(lims)
ax.legend()

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Opex Non-fuel ($/MWh) plant_fuel')
ax.set_ylabel('Opex Non-fuel ($/MWh) plant_unit')
ax.set_title(f"Non-Fuel Opex plant_fuel vs. plant_unit {year} (Non-matching)", {'fontsize': 18,'fontweight' : 'bold'})